# 05 demo

**in**: rendered videos from 03_overlay  
**out**: demo clips ready for presentation

this notebook identifies best moments, cuts highlights, and prepares the final demo package.

In [ ]:
# cell 1: identify best moments (eval swings, goals)

from pathlib import Path
import subprocess

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

OUT_DIR = Path('../data/out')
RENDER_DIR = Path('../data/render')
DEMO_DIR = RENDER_DIR / 'demo'
DEMO_DIR.mkdir(parents=True, exist_ok=True)

# find eval files
eval_files = list(OUT_DIR.glob('*_eval_ts.csv'))
if not eval_files:
    raise FileNotFoundError('no eval_ts.csv found - run 02_eval_bar first')

EVAL_CSV = eval_files[0]
MATCH_NAME = EVAL_CSV.stem.replace('_eval_ts', '')
print(f'analyzing: {MATCH_NAME}')

eval_df = pd.read_csv(EVAL_CSV)
print(f'loaded {len(eval_df)} frames')

# compute eval changes
eval_df['eval_change'] = eval_df['eval_bar'].diff().abs()
eval_df['eval_smooth_change'] = eval_df['eval_bar'].rolling(10).mean().diff().abs()

# find biggest swings
eval_df['is_swing'] = eval_df['eval_smooth_change'] > eval_df['eval_smooth_change'].quantile(0.95)

# cluster swings that are close together
swing_frames = eval_df[eval_df['is_swing']]['frame'].values
swing_times = eval_df[eval_df['is_swing']]['t_sec'].values

moments = []
if len(swing_times) > 0:
    cluster_gap = 10  # seconds
    current_start = swing_times[0]
    current_end = swing_times[0]
    
    for t in swing_times[1:]:
        if t - current_end < cluster_gap:
            current_end = t
        else:
            moments.append({
                'start': max(0, current_start - 5),
                'end': current_end + 5,
                'duration': current_end - current_start + 10,
            })
            current_start = t
            current_end = t
    
    moments.append({
        'start': max(0, current_start - 5),
        'end': current_end + 5,
        'duration': current_end - current_start + 10,
    })

# rank by eval variance in window
for m in moments:
    window_df = eval_df[(eval_df['t_sec'] >= m['start']) & (eval_df['t_sec'] <= m['end'])]
    m['eval_var'] = window_df['eval_bar'].var()
    m['eval_range'] = window_df['eval_bar'].max() - window_df['eval_bar'].min()

moments = sorted(moments, key=lambda x: x['eval_range'], reverse=True)

print(f'\nfound {len(moments)} interesting moments:')
for i, m in enumerate(moments[:10]):
    print(f"  {i+1}. {m['start']:.0f}s - {m['end']:.0f}s (range: {m['eval_range']:.1f})")

In [ ]:
# cell 2: cut 30-60s highlight clips

FINAL_VIDEO = RENDER_DIR / f'{MATCH_NAME}_final.mp4'
if not FINAL_VIDEO.exists():
    # fallback to overlay video
    FINAL_VIDEO = RENDER_DIR / f'{MATCH_NAME}_overlay.mp4'

if not FINAL_VIDEO.exists():
    print(f'ERROR: no rendered video found at {FINAL_VIDEO}')
    print('run 03_overlay first')
else:
    print(f'source video: {FINAL_VIDEO}')


def cut_clip(input_path, output_path, start_sec, duration_sec):
    """Cut a clip from video using ffmpeg."""
    cmd = [
        'ffmpeg', '-y',
        '-ss', str(start_sec),
        '-i', str(input_path),
        '-t', str(duration_sec),
        '-c:v', 'libx264',
        '-preset', 'fast',
        '-crf', '22',
        '-pix_fmt', 'yuv420p',
        str(output_path)
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return result.returncode == 0


# cut top 5 highlight clips
clip_count = min(5, len(moments))
clips = []

for i in range(clip_count):
    m = moments[i]
    
    # target 30-60 seconds
    duration = min(60, max(30, m['duration']))
    start = m['start']
    
    clip_path = DEMO_DIR / f'{MATCH_NAME}_highlight_{i+1:02d}.mp4'
    
    if FINAL_VIDEO.exists():
        success = cut_clip(FINAL_VIDEO, clip_path, start, duration)
        if success:
            clips.append({
                'path': clip_path,
                'start': start,
                'duration': duration,
                'eval_range': m['eval_range'],
            })
            print(f'cut clip {i+1}: {clip_path.name} ({start:.0f}s, {duration:.0f}s)')
        else:
            print(f'failed to cut clip {i+1}')

print(f'\ngenerated {len(clips)} highlight clips')

In [ ]:
# cell 3: add title cards / annotations

import cv2


def add_title_card(video_path, title_text, duration_sec=3):
    """
    Prepend a title card to a video.
    """
    cap = cv2.VideoCapture(str(video_path))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # temp file for title card
    title_path = video_path.parent / f'{video_path.stem}_title.mp4'
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(str(title_path), fourcc, fps, (width, height))
    
    # create title frames
    for _ in range(int(fps * duration_sec)):
        frame = np.zeros((height, width, 3), dtype=np.uint8)
        frame[:] = (30, 30, 30)  # dark gray
        
        # add text
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.5
        thickness = 3
        
        text_size = cv2.getTextSize(title_text, font, font_scale, thickness)[0]
        text_x = (width - text_size[0]) // 2
        text_y = (height + text_size[1]) // 2
        
        cv2.putText(frame, title_text, (text_x, text_y), font, font_scale, 
                    (255, 255, 255), thickness)
        
        writer.write(frame)
    
    writer.release()
    cap.release()
    
    return title_path


def concat_videos(video_paths, output_path):
    """Concatenate multiple videos using ffmpeg."""
    # create concat file
    concat_file = output_path.parent / 'concat.txt'
    with open(concat_file, 'w') as f:
        for p in video_paths:
            f.write(f"file '{p}'\n")
    
    cmd = [
        'ffmpeg', '-y',
        '-f', 'concat',
        '-safe', '0',
        '-i', str(concat_file),
        '-c:v', 'libx264',
        '-preset', 'fast',
        '-crf', '22',
        str(output_path)
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    concat_file.unlink()  # cleanup
    return result.returncode == 0


# add title cards to clips
titled_clips = []

for i, clip in enumerate(clips):
    title = f"Highlight {i+1} - Eval Range: {clip['eval_range']:.0f}"
    
    # create title card
    title_path = add_title_card(clip['path'], title, duration_sec=2)
    
    # concat title + clip
    titled_path = DEMO_DIR / f'{MATCH_NAME}_titled_{i+1:02d}.mp4'
    
    if concat_videos([title_path, clip['path']], titled_path):
        titled_clips.append(titled_path)
        print(f'titled clip: {titled_path.name}')
    
    # cleanup title card
    title_path.unlink(missing_ok=True)

print(f'\ncreated {len(titled_clips)} titled clips')

In [ ]:
# cell 4: export final demo package

# create combined demo reel
DEMO_REEL = DEMO_DIR / f'{MATCH_NAME}_demo_reel.mp4'

if len(titled_clips) > 0:
    # add intro title
    intro_frame = np.zeros((720, 1280, 3), dtype=np.uint8)
    intro_frame[:] = (20, 20, 20)
    
    intro_path = DEMO_DIR / 'intro.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(str(intro_path), fourcc, 25, (1280, 720))
    
    for _ in range(100):  # 4 seconds
        frame = intro_frame.copy()
        
        # title
        cv2.putText(frame, 'BOTTLEJOB DETECTOR', (280, 300), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)
        cv2.putText(frame, 'Soccer Analytics Demo', (400, 380), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (150, 150, 150), 2)
        cv2.putText(frame, f'Match: {MATCH_NAME}', (480, 450), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (100, 100, 100), 2)
        
        writer.write(frame)
    
    writer.release()
    
    # concat all
    all_clips = [intro_path] + titled_clips
    
    if concat_videos(all_clips, DEMO_REEL):
        print(f'demo reel created: {DEMO_REEL}')
    else:
        print('failed to create demo reel')
    
    # cleanup intro
    intro_path.unlink(missing_ok=True)
else:
    print('no clips to combine into demo reel')

# summary
print('\n' + '='*60)
print('DEMO PACKAGE SUMMARY')
print('='*60)
print(f'match: {MATCH_NAME}')
print(f'clips: {len(clips)}')
print(f'demo reel: {DEMO_REEL if DEMO_REEL.exists() else "not created"}')
print('\nfiles:')
for f in sorted(DEMO_DIR.glob('*.mp4')):
    size_mb = f.stat().st_size / (1024 * 1024)
    print(f'  {f.name} ({size_mb:.1f} MB)')

In [ ]:
# cell 5: narration points

print('\n' + '='*60)
print('NARRATION POINTS FOR DEMO')
print('='*60)

print('''
1. INTRO (0:00-0:10)
   - "This is Bottlejob Detector, a soccer analytics system"
   - "It tracks players, computes pitch control, and predicts game momentum"

2. EVAL BAR EXPLANATION (first clip)
   - "The eval bar shows which team is dominating"
   - "Blue = home team advantage, Red = away team advantage"
   - "Range is -100 to +100"

3. PITCH CONTROL (visible in overlay)
   - "The colored regions show which team controls each area"
   - "Based on Voronoi diagrams from player positions"

4. MOMENTUM SHIFTS (highlight clips)
   - "Watch the eval bar swing as possession changes"
   - "Big swings often precede goals or chances"

5. TECHNICAL NOTES
   - Player tracking: YOLO + ByteTrack
   - Pitch mapping: Keypoint detection + homography
   - Eval formula: pitch_control + xT + pressure
   - Pass success: logistic regression on distance/angle/defenders
''')

# eval bar stats for narration
print('\nKEY STATS:')
print(f"  Eval range: {eval_df['eval_bar'].min():.0f} to {eval_df['eval_bar'].max():.0f}")
print(f"  Mean eval: {eval_df['eval_bar'].mean():.1f}")
print(f"  Std dev: {eval_df['eval_bar'].std():.1f}")
print(f"  Big swings (>20 in 10 frames): {(eval_df['eval_change'].rolling(10).sum() > 20).sum()}")

## verification checklist

- [ ] clips are 30-60s each
- [ ] eval bar clearly visible in clips
- [ ] demo reel plays smoothly
- [ ] narration points cover key features